# CRPBot Model Evaluation

**Models**: 3 LSTM (BTC, ETH, SOL) - 128/3/bidirectional, 50 features  
**Goal**: Evaluate against promotion gates (≥68% accuracy, ≤5% calibration error)  
**Time**: ~5-10 minutes on GPU  
**Files**: `/My Drive/crpbot/`

## Step 1: Check GPU

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU not found! Runtime → Change runtime type → GPU")

## Step 2: Install Dependencies

In [ ]:
!pip install -q loguru pandas pyarrow scikit-learn

## Step 3: Mount Google Drive & Verify Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
drive_path = '/content/drive/MyDrive/crpbot'

print(f"\n📂 Checking: {drive_path}")
print(f"Exists: {os.path.exists(drive_path)}\n")

if os.path.exists(drive_path):
    print("Contents:")
    !ls -lh /content/drive/MyDrive/crpbot/
else:
    print("❌ ERROR: Folder not found!")
    print("Expected: /My Drive/crpbot/")

## Step 4: Copy Files to Workspace

In [ ]:
!mkdir -p models/new data/features

# Try multiple folder structures
print("📦 Copying models...")
!cp /content/drive/MyDrive/crpbot/models/*.pt models/new/ 2>/dev/null || \
 cp /content/drive/MyDrive/crpbot/*.pt models/new/ 2>/dev/null || \
 echo "⚠️ No .pt files found"

print("\n📦 Copying features...")
!cp /content/drive/MyDrive/crpbot/features/*.parquet data/features/ 2>/dev/null || \
 cp /content/drive/MyDrive/crpbot/*.parquet data/features/ 2>/dev/null || \
 echo "⚠️ No .parquet files found"

# Verify
import glob
models = glob.glob('models/new/*.pt')
features = glob.glob('data/features/*.parquet')

print(f"\n✅ Ready:")
print(f"  Models: {len(models)}/3")
print(f"  Features: {len(features)}/3")

if len(models) == 3 and len(features) == 3:
    print("\n🎉 All files copied successfully!")
else:
    print("\n⚠️ Missing files. Check your Drive structure.")
    print("\nModels:")
    !ls -lh models/new/
    print("\nFeatures:")
    !ls -lh data/features/

## Step 5: Create Code Modules

In [ ]:
!mkdir -p apps/trainer/{models,train} libs

In [ ]:
%%writefile apps/__init__.py
# Empty

In [ ]:
%%writefile apps/trainer/__init__.py
# Empty

In [ ]:
%%writefile apps/trainer/models/__init__.py
# Empty

In [ ]:
%%writefile apps/trainer/train/__init__.py
# Empty

In [ ]:
%%writefile apps/trainer/models/lstm.py
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int = 128, num_layers: int = 3, dropout: float = 0.2, bidirectional: bool = True):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0, bidirectional=bidirectional)
        lstm_out_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc1 = nn.Linear(lstm_out_size, 64)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out, _ = self.lstm(x)
        x = torch.relu(self.fc1(out[:, -1, :]))
        return self.sigmoid(self.fc2(self.dropout(x)))

In [ ]:
%%writefile apps/trainer/data_pipeline.py
from pathlib import Path
import pandas as pd
from loguru import logger

def load_features_from_parquet(symbol: str, features_dir: str = "data/features"):
    patterns = [f"features_{symbol}_1m_latest.parquet", f"features_{symbol}_1m_*_50feat.parquet", f"features_{symbol}_*.parquet"]
    for pattern in patterns:
        files = list(Path(features_dir).glob(pattern))
        if files:
            path = sorted(files)[-1]
            df = pd.read_parquet(path)
            logger.info(f"Loaded {path.name}: {len(df):,} rows, {len(df.columns)} cols")
            return df
    raise FileNotFoundError(f"No features for {symbol}")

def create_walk_forward_splits(df, train_ratio=0.7, val_ratio=0.15):
    n = len(df)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    train_df = df.iloc[:train_end].copy()
    val_df = df.iloc[train_end:val_end].copy()
    test_df = df.iloc[val_end:].copy()
    logger.info(f"Splits: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")
    return train_df, val_df, test_df

In [ ]:
%%writefile apps/trainer/features.py
import pandas as pd
from loguru import logger

FEATURE_COLUMNS = [
    "session_tokyo", "session_london", "session_ny", "day_of_week", "is_weekend",
    "spread", "spread_pct", "atr", "spread_atr_ratio",
    "volume_ma", "volume_ratio", "volume_trend",
    "sma_7", "sma_14", "sma_21", "sma_50", "price_to_sma_7", "price_to_sma_14", "price_to_sma_21", "price_to_sma_50",
    "rsi", "macd", "macd_signal", "macd_diff", "bb_upper", "bb_middle", "bb_lower", "bb_width",
    "5m_open", "5m_high", "5m_low", "5m_close", "5m_volume",
    "15m_open", "15m_high", "15m_low", "15m_close", "15m_volume",
    "1h_open", "1h_high", "1h_low", "1h_close", "1h_volume",
    "cross_tf_alignment_score", "cross_tf_alignment_direction", "cross_tf_alignment_strength",
    "atr_percentile",
]

def normalize_features(train_df, val_df=None, test_df=None):
    cols = [c for c in FEATURE_COLUMNS if c in train_df.columns]
    stats = {"mean": train_df[cols].mean().to_dict(), "std": train_df[cols].std().to_dict()}
    for col in cols:
        mean, std = stats["mean"][col], stats["std"][col]
        if std > 0:
            train_df[col] = (train_df[col] - mean) / std
            if val_df is not None: val_df[col] = (val_df[col] - mean) / std
            if test_df is not None: test_df[col] = (test_df[col] - mean) / std
    logger.info(f"Normalized {len(cols)} features")
    return train_df, val_df, test_df, stats

In [ ]:
%%writefile apps/trainer/train/dataset.py
import numpy as np
import torch
from torch.utils.data import Dataset
from loguru import logger

class SequenceDataset(Dataset):
    def __init__(self, df, feature_columns, sequence_length=60, prediction_horizon=15, prediction_type="direction"):
        self.df = df.reset_index(drop=True)
        self.features = df[feature_columns].values.astype(np.float32)
        self.sequence_length = sequence_length
        future_close = df["close"].shift(-prediction_horizon)
        current_close = df["close"]
        self.targets = (future_close > current_close).astype(np.float32).values
        self.valid_indices = np.arange(sequence_length, len(df) - prediction_horizon)
        logger.info(f"Dataset: {len(self.valid_indices)} sequences, {len(feature_columns)} features")
    
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        i = self.valid_indices[idx]
        seq = self.features[i - self.sequence_length:i]
        target = self.targets[i]
        return torch.from_numpy(seq), torch.tensor([target], dtype=torch.float32)

## Step 6: Run Evaluation

In [ ]:
%%writefile evaluate.py
import numpy as np
import pandas as pd
import torch
from loguru import logger
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader
from apps.trainer.models.lstm import LSTMModel
from apps.trainer.data_pipeline import load_features_from_parquet, create_walk_forward_splits
from apps.trainer.features import normalize_features, FEATURE_COLUMNS
from apps.trainer.train.dataset import SequenceDataset

def evaluate_model(model_path, symbol, device="cuda", batch_size=64):
    logger.info(f"\n{'='*60}\n🚀 {symbol}\n{'='*60}")
    df = load_features_from_parquet(symbol)
    train_df, val_df, test_df = create_walk_forward_splits(df)
    train_df, val_df, test_df, stats = normalize_features(train_df, val_df, test_df)
    feature_cols = [c for c in FEATURE_COLUMNS if c in test_df.columns]
    test_dataset = SequenceDataset(test_df, feature_columns=feature_cols)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=(device=="cuda"))
    model = LSTMModel(input_size=len(feature_cols), hidden_size=128, num_layers=3, dropout=0.2, bidirectional=True)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model = model.to(device).eval()
    logger.info(f"Evaluating {len(test_dataset):,} sequences...")
    all_preds, all_targets, all_probs = [], [], []
    with torch.no_grad():
        for batch_idx, (seqs, tgts) in enumerate(test_loader):
            outs = model(seqs.to(device))
            probs = outs.squeeze().cpu().numpy()
            preds = (probs >= 0.5).astype(int)
            all_probs.extend(probs.tolist() if probs.ndim > 0 else [probs.item()])
            all_preds.extend(preds.tolist() if preds.ndim > 0 else [preds.item()])
            all_targets.extend(tgts.squeeze().cpu().numpy().tolist())
            if batch_idx % 100 == 0:
                logger.info(f"  {(batch_idx + 1) * batch_size:,}/{len(test_dataset):,}")
    acc = accuracy_score(all_targets, all_preds)
    prec = precision_score(all_targets, all_preds, zero_division=0)
    rec = recall_score(all_targets, all_preds, zero_division=0)
    f1 = f1_score(all_targets, all_preds, zero_division=0)
    bins = np.linspace(0, 1, 11)
    bin_idx = np.digitize(all_probs, bins) - 1
    ece = 0.0
    for i in range(10):
        mask = bin_idx == i
        if mask.sum() > 0:
            ece += mask.sum() / len(all_targets) * abs(np.array(all_targets)[mask].mean() - np.array(all_probs)[mask].mean())
    logger.info(f"\n✅ Results: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}, ECE={ece:.4f}")
    passes = acc >= 0.68 and ece <= 0.05
    logger.info(f"🎯 Gates: Acc≥68% {'✅' if acc >= 0.68 else '❌'} ({acc:.2%}), Cal≤5% {'✅' if ece <= 0.05 else '❌'} ({ece:.2%})")
    logger.info(f"{'✅ PASS' if passes else '❌ FAIL'}\n")
    return {"symbol": symbol, "accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "calibration_error": ece, "num_samples": len(all_targets)}

device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Device: {device}")
results = {}
for symbol in ["BTC-USD", "ETH-USD", "SOL-USD"]:
    model_path = f"models/new/lstm_{symbol.replace('-', '_')}_1m_7b5f0829.pt"
    results[symbol] = evaluate_model(model_path, symbol, device=device)
df = pd.DataFrame(results).T
print(f"\n\n{'='*60}\n📊 SUMMARY\n{'='*60}\n{df.to_string()}\n")
df.to_csv("results.csv")
logger.info("Saved: results.csv")

In [ ]:
!python evaluate.py

## Step 7: Download Results

In [ ]:
from google.colab import files
files.download('results.csv')
print("\n✅ Downloaded results.csv")
print("\nShare this file in the Claude chat for analysis!")